In [1]:
# Dependencies and Setup
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from keys import ykey
import numpy as np


In [2]:
# Call up API key, define headers params for 2 calls

api_key = ykey
headers = {'Authorization': 'Bearer %s' % api_key}

params = {'categories':'kids_activities','location':'Toronto-ON','limit':'50'}
params2 = {'categories':'parks','location':'Toronto-ON','limit':'50'}


In [3]:
# Setup base url, set up requests , print the status codes

url = "https://api.yelp.com/v3/businesses/search?"

req=requests.get(url, params=params, headers=headers)
req2=requests.get(url, params=params2, headers=headers)

print('The status code is {}'.format(req.status_code))


The status code is 200


In [4]:
print('The status code is {}'.format(req2.status_code))

The status code is 200


In [5]:
# Printing the text from the responses
results= json.loads(req.text)
results

{'businesses': [{'id': 'oj7_XBscin51DzpNIVXJqw',
   'alias': 'gymboree-play-and-music-toronto-toronto',
   'name': 'Gymboree Play & Music, Toronto',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/zL0nA1Vie_ErcSEm0fDlbg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/gymboree-play-and-music-toronto-toronto?adjust_creative=F8OOUfRx1ujSudXJ-CRTfA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=F8OOUfRx1ujSudXJ-CRTfA',
   'review_count': 2,
   'categories': [{'alias': 'kids_activities', 'title': 'Kids Activities'},
    {'alias': 'preschools', 'title': 'Preschools'}],
   'rating': 5.0,
   'coordinates': {'latitude': 43.69697, 'longitude': -79.37147},
   'transactions': [],
   'location': {'address1': '1213 Bayview Ave',
    'address2': None,
    'address3': None,
    'city': 'Toronto',
    'zip_code': 'M4G 2Z8',
    'country': 'CA',
    'state': 'ON',
    'display_address': ['1213 Bayview Ave', 'Toronto, ON M4G 2Z8', 'Canada']},
   'phone':

In [6]:
results2= json.loads(req2.text)
results2

{'businesses': [{'id': 'VEMX1R4xtF5AXwKlNMyDVg',
   'alias': 'high-park-toronto',
   'name': 'High Park',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/cpXnEm_O39JX0o5VDe5JDg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/high-park-toronto?adjust_creative=F8OOUfRx1ujSudXJ-CRTfA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=F8OOUfRx1ujSudXJ-CRTfA',
   'review_count': 131,
   'categories': [{'alias': 'parks', 'title': 'Parks'},
    {'alias': 'zoos', 'title': 'Zoos'},
    {'alias': 'playgrounds', 'title': 'Playgrounds'}],
   'rating': 4.5,
   'coordinates': {'latitude': 43.6476281585089,
    'longitude': -79.463939666748},
   'transactions': [],
   'location': {'address1': '1873 Bloor St W',
    'address2': '',
    'address3': '',
    'city': 'Toronto',
    'zip_code': 'M6P 3K7',
    'country': 'CA',
    'state': 'ON',
    'display_address': ['1873 Bloor St W', 'Toronto, ON M6P 3K7', 'Canada']},
   'phone': '',
   'display_phone': '',

In [7]:
# setup dictionary to later convert to a df

city_business_dict = {
                    'Yelp_ID':[],'Business_Name':[],
                    'Business_type':[],'City':[],
                    'Address':[], 'Zipcode':[],
                    'Latitude':[],'Longitude':[],'url':[]
                        }
city_business_dict2 = {
                    'Yelp_ID':[],'Business_Name':[],
                    'Business_type':[],'City':[],
                    'Address':[], 'Zipcode':[],
                    'Latitude':[],'Longitude':[],'url':[]
                        }

In [8]:
# Explore jsons
Y_ID = results['businesses'][0]['id']
kind = results['businesses'][0]['categories'][0]['title']
lat = results['businesses'][0]['coordinates']['latitude']

Y_ID_2 = results2['businesses'][0]['id']
kind_2 = results2['businesses'][0]['categories'][0]['title']
lat_2 = results2['businesses'][0]['coordinates']['latitude']

Y_ID, kind, lat,Y_ID_2, kind_2, lat_2

('oj7_XBscin51DzpNIVXJqw',
 'Kids Activities',
 43.69697,
 'VEMX1R4xtF5AXwKlNMyDVg',
 'Parks',
 43.6476281585089)

In [9]:
# start loop to exctract values from the json calls and append to dictionary

for r in range(len(results["businesses"])):
        city_business_dict['Yelp_ID'].append(results['businesses'][r]['id'])
        city_business_dict['Business_Name'].append(results['businesses'][r]['name'])
        city_business_dict['Business_type'].append(results['businesses'][r]['categories'][0]['title'])
        city_business_dict['City'].append(results['businesses'][r]["location"]["city"])
        city_business_dict['Address'].append(results["businesses"][r]['location']['address1'])
        city_business_dict['Zipcode'].append(results['businesses'][r]['location']['zip_code'])
        city_business_dict['Latitude'].append(results['businesses'][r]['coordinates']['latitude'])
        city_business_dict['Longitude'].append(results['businesses'][r]['coordinates']['longitude'])
        city_business_dict['url'].append(results['businesses'][r]['url'])
        
    


In [10]:
for r in range(len(results2["businesses"])):
        city_business_dict2['Yelp_ID'].append(results2['businesses'][r]['id'])
        city_business_dict2['Business_Name'].append(results2['businesses'][r]['name'])
        city_business_dict2['Business_type'].append(results2['businesses'][r]['categories'][0]['title'])
        city_business_dict2['City'].append(results2['businesses'][r]["location"]["city"])
        city_business_dict2['Address'].append(results2["businesses"][r]['location']['address1'])
        city_business_dict2['Zipcode'].append(results2['businesses'][r]['location']['zip_code'])
        city_business_dict2['Latitude'].append(results2['businesses'][r]['coordinates']['latitude'])
        city_business_dict2['Longitude'].append(results2['businesses'][r]['coordinates']['longitude'])
        city_business_dict2['url'].append(results2['businesses'][r]['url'])

In [11]:
# Check the dictionary
# city_business_dict
# city_business_dict2

In [12]:
# Turn the dictionary in to a data frame
business_kidPlaces_df = pd.DataFrame(city_business_dict)


In [13]:
# Check df for contents
business_kidPlaces_df.head(1)


,Yelp_ID,Business_Name,Business_type,City,Address,Zipcode,Latitude,Longitude,url
0,oj7_XBscin51DzpNIVXJqw,"Gymboree Play & Music, Toronto",Kids Activities,Toronto,1213 Bayview Ave,M4G 2Z8,43.69697,-79.37147,https://www.yelp.com/biz/gymboree-play-and-mus...


In [14]:
business_kidPlaces_df2 = pd.DataFrame(city_business_dict2)
business_kidPlaces_df2.head(1)

,Yelp_ID,Business_Name,Business_type,City,Address,Zipcode,Latitude,Longitude,url
0,VEMX1R4xtF5AXwKlNMyDVg,High Park,Parks,Toronto,1873 Bloor St W,M6P 3K7,43.647628,-79.46394,https://www.yelp.com/biz/high-park-toronto?adj...


In [15]:
# Create engine database and connect

database_path = "kidPlaces_db.sqlite"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [16]:
# add class for Kids Places
from sqlalchemy.orm import Session
session = Session(bind=engine)
class KPlaces(Base):
    __tablename__ = 'KPlaces'
    id = Column(Integer, primary_key=True)
    Yelp_ID = Column(String)
    Business_Name = Column(String)
    Business_type = Column(String)
    City = Column(String)
    Address = Column(String)
    Zipcode = Column(String)
    Latitude = Column(Float)
    Longitude = Column(Float)
    url = Column(String)
conn = engine.connect()
Base.metadata.create_all(conn) 


# Add rows to the table
for index, item in business_kidPlaces_df.iterrows():
    place = KPlaces(Yelp_ID=item['Yelp_ID'],
                  Business_Name=item['Business_Name'],
                  Business_type=item['Business_type'],
                  City=item['City'],
                  Address=item['Address'],
                  Zipcode=item['Zipcode'],
                  Latitude=item['Latitude'],
                  Longitude=item['Longitude'],
                  url=item['url'],
                 )
    session.add(place)
    session.commit()

In [17]:
# business_kidPlaces_df2.head(1)

In [18]:
# Add rows containg parks infomration
for index, item in business_kidPlaces_df2.iterrows():
    place = KPlaces(Yelp_ID=item['Yelp_ID'],
                  Business_Name=item['Business_Name'],
                  Business_type=item['Business_type'],
                  City=item['City'],
                  Address=item['Address'],
                  Zipcode=item['Zipcode'],
                  Latitude=item['Latitude'],
                  Longitude=item['Longitude'],
                  url=item['url'],
                 )
    session.add(place)
    session.commit()

In [19]:
# Check table the KPlaces table fro contents
engine.execute("SELECT * FROM KPlaces").fetchall()

[(1, 'oj7_XBscin51DzpNIVXJqw', 'Gymboree Play & Music, Toronto', 'Kids Activities', 'Toronto', '1213 Bayview Ave', 'M4G 2Z8', 43.69697, -79.37147, 'https://www.yelp.com/biz/gymboree-play-and-music-toronto-toronto?adjust_creative=F8OOUfRx1ujSudXJ-CRTfA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=F8OOUfRx1ujSudXJ-CRTfA'),
 (2, 'V8n2enXnTFdzF30rBd-BdQ', 'In The Mix Program', 'Kids Activities', 'Toronto', '', '', 43.6487, -79.38545, 'https://www.yelp.com/biz/in-the-mix-program-toronto?adjust_creative=F8OOUfRx1ujSudXJ-CRTfA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=F8OOUfRx1ujSudXJ-CRTfA'),
 (3, '2n4zF45Y75Rx4QS6TWcqRw', 'Jill And The Beanstalk', 'Kids Activities', 'Toronto', '814 College Street', 'M6G 1C8', 43.6546472761219, -79.4220370084964, 'https://www.yelp.com/biz/jill-and-the-beanstalk-toronto?adjust_creative=F8OOUfRx1ujSudXJ-CRTfA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=F8OOUfRx1ujSudXJ-CRTfA'),
 (4,

In [20]:
# Scraping more data

from splinter import Browser
from bs4 import BeautifulSoup as bs
import shutil


In [21]:
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

In [22]:
url_rest = "https://theculturetrip.com/north-america/canada/articles/12-family-friendly-restaurants-in-toronto/"
response = requests.get(url_rest)
print(response)

<Response [200]>


In [23]:
# print(response.text)

In [24]:
soup = bs(response.text,"html.parser")

In [25]:
print (soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8" class="next-head"/>
  <title class="next-head">
   The 12 Best Family Friendly Restaurants In Toronto
  </title>
  <meta class="next-head" content="initial-scale=1.0, width=device-width, maximum-scale=1" name="viewport"/>
  <meta class="next-head" content="The 12 Best Family Friendly Restaurants In Toronto" name="title"/>
  <meta class="next-head" content="The challenges of eating out with family can be excruciating, but these family friendly restaurants in Toronto are here to save the day!" name="description"/>
  <meta class="next-head" content="Maheen Mansoor" name="author"/>
  <meta class="next-head" content="summary_large_image" name="twitter:card"/>
  <meta class="next-head" content="@CultureTrip" name="twitter:site"/>
  <meta class="next-head" content="The 12 Best Family Friendly Restaurants In Toronto" name="twitter:title"/>
  <meta class="next-head" content="The 12 Best Family Friendly Restaurants In Toronto" name="twitter:

In [26]:
restaurant = soup.find_all("h2", class_ = "item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF")
print (restaurant)

[<h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">Barque</h2>, <h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">Terroni</h2>, <h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">The Ace</h2>, <h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">Uncle Betty’s Diner</h2>, <h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">The Old Spaghetti Factory</h2>, <h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">Fresh</h2>, <h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">Hello Darling</h2>]


In [27]:
restautant_dictionary = {'Restaurant_name':[],
                  
                  'Restaurant_address':[]}

In [28]:

for r in restaurant:
    restautant_dictionary['Restaurant_name'].append(r.text)
    print(r)

<h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">Barque</h2>
<h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">Terroni</h2>
<h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">The Ace</h2>
<h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">Uncle Betty’s Diner</h2>
<h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">The Old Spaghetti Factory</h2>
<h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">Fresh</h2>
<h2 class="item-card-title__ItemTitleWrapper-a1kgd2-0 xrcvF">Hello Darling</h2>


In [29]:
restautant_dictionary

{'Restaurant_name': ['Barque',
  'Terroni',
  'The Ace',
  'Uncle Betty’s Diner',
  'The Old Spaghetti Factory',
  'Fresh',
  'Hello Darling'],
 'Restaurant_address': []}

In [30]:
restaurant2 = soup.find_all("h3")
print (restaurant2)

[<h3><span style="text-decoration: underline;">Magic Oven</span></h3>, <h3><span style="text-decoration: underline;">Pizzeria Libretto</span></h3>, <h3><span style="text-decoration: underline;">Fresco Fish &amp; Chips</span></h3>, <h3><span style="text-decoration: underline;">Emma’s Country Kitchen</span></h3>, <h3><span style="text-decoration: underline;">Prohibition Gastrohouse</span></h3>, <h3>The Company</h3>, <h3>Contact Us</h3>, <h3>Legal</h3>, <h3>Follow Us</h3>]


In [31]:
for r in restaurant2[:5]:
    restautant_dictionary['Restaurant_name'].append(r.text)
    print (r)

<h3><span style="text-decoration: underline;">Magic Oven</span></h3>
<h3><span style="text-decoration: underline;">Pizzeria Libretto</span></h3>
<h3><span style="text-decoration: underline;">Fresco Fish &amp; Chips</span></h3>
<h3><span style="text-decoration: underline;">Emma’s Country Kitchen</span></h3>
<h3><span style="text-decoration: underline;">Prohibition Gastrohouse</span></h3>


In [32]:
restautant_dictionary

{'Restaurant_name': ['Barque',
  'Terroni',
  'The Ace',
  'Uncle Betty’s Diner',
  'The Old Spaghetti Factory',
  'Fresh',
  'Hello Darling',
  'Magic Oven',
  'Pizzeria Libretto',
  'Fresco Fish & Chips',
  'Emma’s Country Kitchen',
  'Prohibition Gastrohouse'],
 'Restaurant_address': []}

In [33]:
restaturant_address = ["299 Roncesvalles Ave, Toronto, ON",
                       "720 Queen St W, Toronto, ON",
                       "231A Roncesvalles Ave, Toronto, ON",
                       "2590 Yonge St, Toronto, ON",
                       "54 The Esplanade, Toronto, ON",
                       "147 Spadina Ave, Toronto, ON",
                       "827 Lansdowne Ave, Toronto, ON",
                       "1450 Danforth Avenue, Toronto, ON",
                       "550 Danforth Avenue, Toronto, ON",
                       "816 St Clair Ave W, Toronto, ON",
                       "1108 St Clair Ave W, Toronto, ON",
                       "696 Queen Street E, Toronto, ON",
                      ]

In [34]:
for r in restaturant_address:
        restautant_dictionary['Restaurant_address'].append(r)

In [35]:
restautant_dictionary

{'Restaurant_name': ['Barque',
  'Terroni',
  'The Ace',
  'Uncle Betty’s Diner',
  'The Old Spaghetti Factory',
  'Fresh',
  'Hello Darling',
  'Magic Oven',
  'Pizzeria Libretto',
  'Fresco Fish & Chips',
  'Emma’s Country Kitchen',
  'Prohibition Gastrohouse'],
 'Restaurant_address': ['299 Roncesvalles Ave, Toronto, ON',
  '720 Queen St W, Toronto, ON',
  '231A Roncesvalles Ave, Toronto, ON',
  '2590 Yonge St, Toronto, ON',
  '54 The Esplanade, Toronto, ON',
  '147 Spadina Ave, Toronto, ON',
  '827 Lansdowne Ave, Toronto, ON',
  '1450 Danforth Avenue, Toronto, ON',
  '550 Danforth Avenue, Toronto, ON',
  '816 St Clair Ave W, Toronto, ON',
  '1108 St Clair Ave W, Toronto, ON',
  '696 Queen Street E, Toronto, ON']}

In [36]:
restaturants_df = pd.DataFrame(restautant_dictionary)
restaturants_df.head(1)

,Restaurant_name,Restaurant_address
0,Barque,"299 Roncesvalles Ave, Toronto, ON"


In [37]:
# Create the class for restaurants
from sqlalchemy.orm import Session
session = Session(bind=engine)
class KRestaurant(Base):
    __tablename__ = 'KRestaurant'
    id = Column(Integer, primary_key=True)
    Restaurant_name = Column(String)
    Restaurant_address = Column(String)
conn = engine.connect()
Base.metadata.create_all(conn) 

# Add rows to the KRestaurant table
for index, item in restaturants_df.iterrows():
    restaurant = KRestaurant(Restaurant_name=item['Restaurant_name'],
                  Restaurant_address=item['Restaurant_address']
                            )
    session.add(restaurant)
    session.commit()

In [38]:
# Check the KRestaurant table
engine.execute("SELECT * FROM KRestaurant").fetchall()

[(1, 'Barque', '299 Roncesvalles Ave, Toronto, ON'),
 (2, 'Terroni', '720 Queen St W, Toronto, ON'),
 (3, 'The Ace', '231A Roncesvalles Ave, Toronto, ON'),
 (4, 'Uncle Betty’s Diner', '2590 Yonge St, Toronto, ON'),
 (5, 'The Old Spaghetti Factory', '54 The Esplanade, Toronto, ON'),
 (6, 'Fresh', '147 Spadina Ave, Toronto, ON'),
 (7, 'Hello Darling', '827 Lansdowne Ave, Toronto, ON'),
 (8, 'Magic Oven', '1450 Danforth Avenue, Toronto, ON'),
 (9, 'Pizzeria Libretto', '550 Danforth Avenue, Toronto, ON'),
 (10, 'Fresco Fish & Chips', '816 St Clair Ave W, Toronto, ON'),
 (11, 'Emma’s Country Kitchen', '1108 St Clair Ave W, Toronto, ON'),
 (12, 'Prohibition Gastrohouse', '696 Queen Street E, Toronto, ON')]

In [39]:
url_city = "https://www.moneysense.ca/spend/real-estate/canadas-best-places-to-live-25-2018/"
response = requests.get(url_city)
print(response)

<Response [200]>


In [40]:
soupc = bs(response.text,"html.parser")
print (soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8" class="next-head"/>
  <title class="next-head">
   The 12 Best Family Friendly Restaurants In Toronto
  </title>
  <meta class="next-head" content="initial-scale=1.0, width=device-width, maximum-scale=1" name="viewport"/>
  <meta class="next-head" content="The 12 Best Family Friendly Restaurants In Toronto" name="title"/>
  <meta class="next-head" content="The challenges of eating out with family can be excruciating, but these family friendly restaurants in Toronto are here to save the day!" name="description"/>
  <meta class="next-head" content="Maheen Mansoor" name="author"/>
  <meta class="next-head" content="summary_large_image" name="twitter:card"/>
  <meta class="next-head" content="@CultureTrip" name="twitter:site"/>
  <meta class="next-head" content="The 12 Best Family Friendly Restaurants In Toronto" name="twitter:title"/>
  <meta class="next-head" content="The 12 Best Family Friendly Restaurants In Toronto" name="twitter:

In [41]:
city = soupc.find_all("h2")[1].text
print (city)

25. Whitby, Ont.


In [42]:
type(city)

str

In [43]:
# Will use this later to lop and create a df i will hard code for the moment
import re
x = re.sub("\.", " ", city)
print(x)

scity =  x.split(' ',1)

scity[0]

25  Whitby, Ont 


'25'

In [44]:
cities = soupc.find_all("h2")
cities

[<h2 class="post-excerpt" id="post-excerpt" itemprop="description">The No. 1 spot has a strong economy, low crime rates and great weather</h2>,
 <h2>25. Whitby, Ont.</h2>,
 <h2>24. New Tecumseth, Ont.</h2>,
 <h2>23. Newmarket, Ont.</h2>,
 <h2>22. Bonnyville No. 87, Alta.</h2>,
 <h2>21. The Nation, Ont.</h2>,
 <h2>20. Whistler, B.C.</h2>,
 <h2>19. St. Albert, Alta.</h2>,
 <h2>18. King, Ont.</h2>,
 <h2>17. Lévis, Que.</h2>,
 <h2>16. Toronto, Ont.</h2>,
 <h2>15. Fort St. John, B.C.</h2>,
 <h2>14. Saugeen Shores, Ont.</h2>,
 <h2>13. Mont-Royal, Que.</h2>,
 <h2>12. Red Deer, Alta.</h2>,
 <h2>11. Camrose, Alta.</h2>,
 <h2>10. Halton Hills, Ont.</h2>,
 <h2>9. Saint-Lambert, Que.</h2>,
 <h2>8. Westmount, Que.</h2>,
 <h2>7. Canmore, Alta.</h2>,
 <h2>6. Milton, Ont.</h2>,
 <h2>5. Lacombe, Alta.</h2>,
 <h2>4. Saint-Bruno-de-Montarville, Que.</h2>,
 <h2>3. Russell Township, Ont.</h2>,
 <h2>2. Ottawa, Ont.</h2>,
 <h2>1. Oakville, Ont.</h2>]

In [45]:
City_rank_dictionary = {
    "Rank":[],
    "City":[]
}

In [46]:
split_Citylist = []
for city in cities[1:]:
    split_Citylist.append(re.sub("\.", " ", city.text).split(' ',1))

In [47]:
r = [int(i[0]) for i in split_Citylist]
c = [i[1] for i in split_Citylist]

In [48]:
City_rank_dictionary["Rank"] = r
City_rank_dictionary["City"] = c

In [49]:
# City_rank_dictionary

In [50]:
CityRanks_df = pd.DataFrame(City_rank_dictionary)
Cities_rank_df = CityRanks_df.sort_index(ascending=False).reset_index(drop=True)


In [51]:
Cities_rank_df.head(1)

,Rank,City
0,1,"Oakville, Ont"


In [52]:
# Create Crank class
from sqlalchemy.orm import Session
session = Session(bind=engine)
class Crank(Base):
    __tablename__ = 'Crank'
    id = Column(Integer, primary_key=True)
    Rank = Column(Integer)
    City = Column(String)
conn = engine.connect()
Base.metadata.create_all(conn) 

# Add rows to the Crank table
for index, item in Cities_rank_df.iterrows():
    city = Crank(Rank=item['Rank'],
                  City=item['City']
                            )
    session.add(city)
    session.commit()

In [53]:
# Check the Crank table
engine.execute("SELECT * FROM Crank").fetchall()

[(1, 1, ' Oakville, Ont '),
 (2, 2, ' Ottawa, Ont '),
 (3, 3, ' Russell Township, Ont '),
 (4, 4, ' Saint-Bruno-de-Montarville, Que '),
 (5, 5, ' Lacombe, Alta '),
 (6, 6, ' Milton, Ont '),
 (7, 7, ' Canmore, Alta '),
 (8, 8, ' Westmount, Que '),
 (9, 9, ' Saint-Lambert, Que '),
 (10, 10, ' Halton Hills, Ont '),
 (11, 11, ' Camrose, Alta '),
 (12, 12, ' Red Deer, Alta '),
 (13, 13, ' Mont-Royal, Que '),
 (14, 14, ' Saugeen Shores, Ont '),
 (15, 15, ' Fort St  John, B C '),
 (16, 16, ' Toronto, Ont '),
 (17, 17, ' Lévis, Que '),
 (18, 18, ' King, Ont '),
 (19, 19, ' St  Albert, Alta '),
 (20, 20, ' Whistler, B C '),
 (21, 21, ' The Nation, Ont '),
 (22, 22, ' Bonnyville No  87, Alta '),
 (23, 23, ' Newmarket, Ont '),
 (24, 24, ' New Tecumseth, Ont '),
 (25, 25, ' Whitby, Ont ')]

In [54]:

# Check reflect

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [55]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['Crank', 'KPlaces', 'KRestaurant']

In [56]:
Ctrank = Base.classes.Crank
first_row = session.query(Ctrank).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x177b35b9f98>,
 'City': ' Oakville, Ont ',
 'Rank': 1,
 'id': 1}

In [57]:
Kp = Base.classes.KPlaces
first_row = session.query(Kp).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x177b35e7940>,
 'Longitude': -79.37147,
 'Zipcode': 'M4G 2Z8',
 'City': 'Toronto',
 'Business_Name': 'Gymboree Play & Music, Toronto',
 'id': 1,
 'url': 'https://www.yelp.com/biz/gymboree-play-and-music-toronto-toronto?adjust_creative=F8OOUfRx1ujSudXJ-CRTfA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=F8OOUfRx1ujSudXJ-CRTfA',
 'Latitude': 43.69697,
 'Address': '1213 Bayview Ave',
 'Business_type': 'Kids Activities',
 'Yelp_ID': 'oj7_XBscin51DzpNIVXJqw'}

In [58]:
Kr = Base.classes.KRestaurant
first_row = session.query(Kr).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x177b35ec0f0>,
 'Restaurant_name': 'Barque',
 'Restaurant_address': '299 Roncesvalles Ave, Toronto, ON',
 'id': 1}